In [1]:
import pickle
import numpy as np
import pandas as pd

from datetime import datetime as dt
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from category_encoders.one_hot import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

Сохраняем необходимые данные (чтобы не качать при каждом перезапуске ноутбука).

In [2]:
# cкачиваем все данные из post_text
post_text = pd.read_sql(
    "SELECT * FROM public.post_text_df",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем post_text в файл
post_text.to_csv('post_text.csv', index=False)

In [3]:
# скачиваем среднее по target по каждому post_id из feed_data
post_stat = pd.read_sql(
    "SELECT post_id, AVG(target) AS post_stat FROM public.feed_data GROUP BY post_id",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем post_text в файл
post_stat.to_csv('post_stat.csv', index=False)

In [4]:
# cкачиваем все данные из user_data
user_data = pd.read_sql(
    "SELECT * FROM public.user_data",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем user_data в файл
user_data.to_csv('user_data.csv', index=False)

In [5]:
# скачиваем среднее по target по каждому user_id из feed_data
user_stat = pd.read_sql(
    "SELECT user_id, AVG(target) AS user_stat FROM public.feed_data GROUP BY user_id",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем user_data в файл
user_stat.to_csv('user_stat.csv', index=False)

In [6]:
# скачиваем последние 100 тысяч записей в feed_data
feed_data = pd.read_sql(
    "SELECT * FROM public.feed_data WHERE action = 'view' ORDER BY timestamp DESC LIMIT 100000",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем feed_data в файл
feed_data.to_csv('feed_data.csv', index=False)

Загружаем данные из ранее сохранённых файлов

In [7]:
post_text = pd.read_csv('post_text.csv')

In [8]:
post_stat = pd.read_csv('post_stat.csv')

In [9]:
user_data = pd.read_csv('user_data.csv')

In [10]:
user_stat = pd.read_csv('user_stat.csv')

In [11]:
feed_data = pd.read_csv('feed_data.csv')
feed_data.sort_values('timestamp', inplace=True, ignore_index=True)

В таблице feed_data есть колонка 'target', она будет являться целевой в нашем обучении.

0 - пост не получил like от пользователя, которому был показан.

1 - пост получил like.

При помощи метода TF-IDF в таблице post_text преобразуем текстовую колонку 'text' в векторный вид.
Из получившейся матрицы возьмём построчные максимум.

In [12]:
post_text_copy = post_text.copy()

vectorizer = TfidfVectorizer()
F = vectorizer.fit_transform(post_text['text'])
post_text_copy['text'] = F.toarray().max(axis=1)

При помощи метода главных компонентов понизим размерность полученной выше матрицы векторов текстов и добавим как признаки.

In [13]:
n = 5
columns = [f"text_{i}th_cluster" for i in range(1, n+1)]

pca = PCA(n_components=n, random_state=1)
pca_decomp = pca.fit_transform(F.toarray())

pca_decomp = pd.DataFrame(pca_decomp, columns=columns)

post_text_copy = pd.concat((post_text_copy, pca_decomp), axis=1)

Соединяем таблицы по колонкам post_id и user_id.

In [14]:
df = pd.merge(feed_data,
              post_text_copy,
              on='post_id',
              how='left')

In [15]:
df = pd.merge(df,
              post_stat,
              on='post_id',
              how='left')

In [16]:
df = pd.merge(df,
              user_data,
              on='user_id',
              how='left')

In [17]:
df = pd.merge(df,
              user_stat,
              on='user_id',
              how='left')

Из колонки timestamp выделяем дополнительные признаки: год, месяц, день, час, минута, секунда. А исходную колонку удаляем.

In [18]:
df['timestamp'] = df['timestamp'].astype('datetime64')

df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
df['second'] = df['timestamp'].dt.second

df = df.drop('timestamp', axis=1)

Делим сгруппированый набор данных на признаки "X" и целевую колонку "Y".

In [19]:
X = df.drop(['user_id', 'post_id', 'action', 'target'], axis=1)
y = df['target']

Делим колонки в таблице признаков на числовые и категориальные, последние в свою очередь делим на признаки, которые будут обрабатываться методом One-Hot или кодирования по таргету.

In [20]:
numeric_columns = list(X.select_dtypes(exclude='object').columns)
categorical_columns = list(X.select_dtypes(include='object').columns)

columns_ohe = [x for x in categorical_columns if X[x].nunique() < 5]
columns_mte = [x for x in categorical_columns if X[x].nunique() >= 5]

numeric_columns_ind = [list(X.columns).index(col) for col in numeric_columns]
columns_ohe_ind = [list(X.columns).index(col) for col in columns_ohe]
columns_mte_ind = [list(X.columns).index(col) for col in columns_mte]

Создаём трансформер, который будет преобразовывать данные признаков следующим образом: числовые признаки стандартизирует, а к категориальным применит метод One-Hot и TargetEncoder.

In [21]:
t = [('StandardScaler', StandardScaler(), numeric_columns_ind),
     ('OneHotEncoder', OneHotEncoder(), columns_ohe_ind),
     ('MeanTargetEncoder', TargetEncoder(), columns_mte_ind)]

columns_transformer = ColumnTransformer(transformers=t)

D:\anaconda3\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
D:\anaconda3\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


Делим данные на тренеровочные и тестовые.

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=1)

Проверяем баланс классов на тренеровке и на тесте.

In [23]:
y_train.value_counts()[1] / len(y_train)

0.1572125

In [24]:
y_test.value_counts()[1] / len(y_test)

0.1559

Стоит отметить, что в данных, на которых выполняется обучение, целевые классы имеют явный дисбаланс: класс 1 порядка 16%. При разбивке данных на тренеровочные и тестывые соотношение классов будем считать приемлемым.

При помощи Pipeline проводим обучение модели CatBoostClassifier. Параметры модели были подобранны вручную.

In [25]:
%%time

pipe = Pipeline([('columns_transformer', columns_transformer),
                 ('decision_tree', CatBoostClassifier(iterations = 1750,
                                                      loss_function = 'MultiClass',
                                                      early_stopping_rounds=10,
                                                      thread_count = 100,
                                                      random_state=1))])
pipe.fit(X_train, y_train)

D:\anaconda3\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
D:\anaconda3\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


Learning rate set to 0.061661
0:	learn: 0.6644872	total: 160ms	remaining: 4m 39s
1:	learn: 0.6388205	total: 172ms	remaining: 2m 30s
2:	learn: 0.6161110	total: 183ms	remaining: 1m 46s
3:	learn: 0.5959765	total: 197ms	remaining: 1m 25s
4:	learn: 0.5780793	total: 211ms	remaining: 1m 13s
5:	learn: 0.5620042	total: 223ms	remaining: 1m 4s
6:	learn: 0.5477478	total: 234ms	remaining: 58.4s
7:	learn: 0.5348844	total: 247ms	remaining: 53.8s
8:	learn: 0.5234482	total: 260ms	remaining: 50.2s
9:	learn: 0.5130075	total: 272ms	remaining: 47.3s
10:	learn: 0.5035969	total: 284ms	remaining: 45s
11:	learn: 0.4948834	total: 298ms	remaining: 43.2s
12:	learn: 0.4869697	total: 311ms	remaining: 41.5s
13:	learn: 0.4800752	total: 323ms	remaining: 40s
14:	learn: 0.4738772	total: 335ms	remaining: 38.8s
15:	learn: 0.4680847	total: 348ms	remaining: 37.7s
16:	learn: 0.4630105	total: 361ms	remaining: 36.8s
17:	learn: 0.4585276	total: 376ms	remaining: 36.1s
18:	learn: 0.4542725	total: 387ms	remaining: 35.3s
19:	learn:

168:	learn: 0.3911986	total: 2.15s	remaining: 20.2s
169:	learn: 0.3909854	total: 2.17s	remaining: 20.1s
170:	learn: 0.3907090	total: 2.18s	remaining: 20.1s
171:	learn: 0.3906507	total: 2.19s	remaining: 20.1s
172:	learn: 0.3905275	total: 2.2s	remaining: 20.1s
173:	learn: 0.3904764	total: 2.21s	remaining: 20s
174:	learn: 0.3901962	total: 2.22s	remaining: 20s
175:	learn: 0.3900939	total: 2.24s	remaining: 20s
176:	learn: 0.3898731	total: 2.25s	remaining: 20s
177:	learn: 0.3898415	total: 2.26s	remaining: 19.9s
178:	learn: 0.3895096	total: 2.27s	remaining: 19.9s
179:	learn: 0.3892450	total: 2.28s	remaining: 19.9s
180:	learn: 0.3889866	total: 2.29s	remaining: 19.9s
181:	learn: 0.3888055	total: 2.3s	remaining: 19.8s
182:	learn: 0.3887392	total: 2.31s	remaining: 19.8s
183:	learn: 0.3886929	total: 2.33s	remaining: 19.8s
184:	learn: 0.3886394	total: 2.34s	remaining: 19.8s
185:	learn: 0.3882715	total: 2.35s	remaining: 19.7s
186:	learn: 0.3881063	total: 2.36s	remaining: 19.7s
187:	learn: 0.3880580	

343:	learn: 0.3716501	total: 4.13s	remaining: 16.9s
344:	learn: 0.3714662	total: 4.14s	remaining: 16.9s
345:	learn: 0.3713788	total: 4.16s	remaining: 16.9s
346:	learn: 0.3713094	total: 4.17s	remaining: 16.9s
347:	learn: 0.3712059	total: 4.18s	remaining: 16.8s
348:	learn: 0.3711200	total: 4.19s	remaining: 16.8s
349:	learn: 0.3710735	total: 4.2s	remaining: 16.8s
350:	learn: 0.3709270	total: 4.21s	remaining: 16.8s
351:	learn: 0.3708930	total: 4.22s	remaining: 16.8s
352:	learn: 0.3708668	total: 4.24s	remaining: 16.8s
353:	learn: 0.3708336	total: 4.25s	remaining: 16.7s
354:	learn: 0.3707232	total: 4.26s	remaining: 16.7s
355:	learn: 0.3706803	total: 4.27s	remaining: 16.7s
356:	learn: 0.3706441	total: 4.28s	remaining: 16.7s
357:	learn: 0.3705232	total: 4.29s	remaining: 16.7s
358:	learn: 0.3704604	total: 4.3s	remaining: 16.7s
359:	learn: 0.3704024	total: 4.31s	remaining: 16.7s
360:	learn: 0.3701661	total: 4.32s	remaining: 16.6s
361:	learn: 0.3701313	total: 4.34s	remaining: 16.6s
362:	learn: 0.

519:	learn: 0.3597757	total: 6.11s	remaining: 14.4s
520:	learn: 0.3597300	total: 6.12s	remaining: 14.4s
521:	learn: 0.3597101	total: 6.13s	remaining: 14.4s
522:	learn: 0.3596798	total: 6.14s	remaining: 14.4s
523:	learn: 0.3596509	total: 6.15s	remaining: 14.4s
524:	learn: 0.3595045	total: 6.16s	remaining: 14.4s
525:	learn: 0.3594692	total: 6.17s	remaining: 14.4s
526:	learn: 0.3593588	total: 6.18s	remaining: 14.3s
527:	learn: 0.3593266	total: 6.19s	remaining: 14.3s
528:	learn: 0.3592917	total: 6.2s	remaining: 14.3s
529:	learn: 0.3591794	total: 6.21s	remaining: 14.3s
530:	learn: 0.3591383	total: 6.23s	remaining: 14.3s
531:	learn: 0.3591022	total: 6.24s	remaining: 14.3s
532:	learn: 0.3589626	total: 6.25s	remaining: 14.3s
533:	learn: 0.3588896	total: 6.26s	remaining: 14.3s
534:	learn: 0.3588631	total: 6.27s	remaining: 14.2s
535:	learn: 0.3588071	total: 6.28s	remaining: 14.2s
536:	learn: 0.3587689	total: 6.29s	remaining: 14.2s
537:	learn: 0.3587237	total: 6.3s	remaining: 14.2s
538:	learn: 0.

...

1197:	learn: 0.3269193	total: 13.8s	remaining: 6.34s
1198:	learn: 0.3268978	total: 13.8s	remaining: 6.33s
1199:	learn: 0.3268759	total: 13.8s	remaining: 6.32s
1200:	learn: 0.3268543	total: 13.8s	remaining: 6.3s
1201:	learn: 0.3268288	total: 13.8s	remaining: 6.29s
1202:	learn: 0.3268074	total: 13.8s	remaining: 6.28s
1203:	learn: 0.3267779	total: 13.8s	remaining: 6.27s
1204:	learn: 0.3266153	total: 13.8s	remaining: 6.26s
1205:	learn: 0.3265996	total: 13.8s	remaining: 6.25s
1206:	learn: 0.3265786	total: 13.9s	remaining: 6.23s
1207:	learn: 0.3265546	total: 13.9s	remaining: 6.22s
1208:	learn: 0.3265325	total: 13.9s	remaining: 6.21s
1209:	learn: 0.3265134	total: 13.9s	remaining: 6.2s
1210:	learn: 0.3264872	total: 13.9s	remaining: 6.18s
1211:	learn: 0.3264674	total: 13.9s	remaining: 6.17s
1212:	learn: 0.3264441	total: 13.9s	remaining: 6.16s
1213:	learn: 0.3264143	total: 13.9s	remaining: 6.15s
1214:	learn: 0.3263967	total: 13.9s	remaining: 6.14s
1215:	learn: 0.3263776	total: 14s	remaining: 6.1

1357:	learn: 0.3208845	total: 15.6s	remaining: 4.49s
1358:	learn: 0.3208601	total: 15.6s	remaining: 4.48s
1359:	learn: 0.3208475	total: 15.6s	remaining: 4.47s
1360:	learn: 0.3208192	total: 15.6s	remaining: 4.46s
1361:	learn: 0.3207868	total: 15.6s	remaining: 4.44s
1362:	learn: 0.3207531	total: 15.6s	remaining: 4.43s
1363:	learn: 0.3206463	total: 15.6s	remaining: 4.42s
1364:	learn: 0.3206244	total: 15.6s	remaining: 4.41s
1365:	learn: 0.3205921	total: 15.7s	remaining: 4.4s
1366:	learn: 0.3205656	total: 15.7s	remaining: 4.39s
1367:	learn: 0.3205315	total: 15.7s	remaining: 4.38s
1368:	learn: 0.3205144	total: 15.7s	remaining: 4.37s
1369:	learn: 0.3204903	total: 15.7s	remaining: 4.35s
1370:	learn: 0.3204621	total: 15.7s	remaining: 4.34s
1371:	learn: 0.3204319	total: 15.7s	remaining: 4.33s
1372:	learn: 0.3204069	total: 15.7s	remaining: 4.32s
1373:	learn: 0.3203872	total: 15.7s	remaining: 4.31s
1374:	learn: 0.3203587	total: 15.8s	remaining: 4.3s
1375:	learn: 0.3203180	total: 15.8s	remaining: 4

1529:	learn: 0.3147710	total: 17.5s	remaining: 2.52s
1530:	learn: 0.3147506	total: 17.5s	remaining: 2.5s
1531:	learn: 0.3147128	total: 17.5s	remaining: 2.49s
1532:	learn: 0.3146800	total: 17.5s	remaining: 2.48s
1533:	learn: 0.3146655	total: 17.6s	remaining: 2.47s
1534:	learn: 0.3146290	total: 17.6s	remaining: 2.46s
1535:	learn: 0.3145163	total: 17.6s	remaining: 2.45s
1536:	learn: 0.3144984	total: 17.6s	remaining: 2.44s
1537:	learn: 0.3144589	total: 17.6s	remaining: 2.42s
1538:	learn: 0.3144363	total: 17.6s	remaining: 2.41s
1539:	learn: 0.3144255	total: 17.6s	remaining: 2.4s
1540:	learn: 0.3143362	total: 17.6s	remaining: 2.39s
1541:	learn: 0.3143240	total: 17.6s	remaining: 2.38s
1542:	learn: 0.3143047	total: 17.6s	remaining: 2.37s
1543:	learn: 0.3142778	total: 17.7s	remaining: 2.36s
1544:	learn: 0.3142444	total: 17.7s	remaining: 2.34s
1545:	learn: 0.3142252	total: 17.7s	remaining: 2.33s
1546:	learn: 0.3141921	total: 17.7s	remaining: 2.32s
1547:	learn: 0.3141796	total: 17.7s	remaining: 2

1702:	learn: 0.3092622	total: 19.5s	remaining: 538ms
1703:	learn: 0.3092389	total: 19.5s	remaining: 526ms
1704:	learn: 0.3092208	total: 19.5s	remaining: 515ms
1705:	learn: 0.3091946	total: 19.5s	remaining: 503ms
1706:	learn: 0.3091619	total: 19.5s	remaining: 492ms
1707:	learn: 0.3091198	total: 19.5s	remaining: 480ms
1708:	learn: 0.3090968	total: 19.5s	remaining: 469ms
1709:	learn: 0.3090698	total: 19.6s	remaining: 457ms
1710:	learn: 0.3090478	total: 19.6s	remaining: 446ms
1711:	learn: 0.3090116	total: 19.6s	remaining: 435ms
1712:	learn: 0.3089135	total: 19.6s	remaining: 423ms
1713:	learn: 0.3088822	total: 19.6s	remaining: 412ms
1714:	learn: 0.3088608	total: 19.6s	remaining: 400ms
1715:	learn: 0.3088338	total: 19.6s	remaining: 389ms
1716:	learn: 0.3088115	total: 19.6s	remaining: 377ms
1717:	learn: 0.3087450	total: 19.7s	remaining: 366ms
1718:	learn: 0.3087178	total: 19.7s	remaining: 355ms
1719:	learn: 0.3087007	total: 19.7s	remaining: 343ms
1720:	learn: 0.3086743	total: 19.7s	remaining:

Pipeline(steps=[('columns_transformer',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  [0, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   12, 15, 16, 17, 18, 19, 20,
                                                   21]),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(), [13, 14]),
                                                 ('MeanTargetEncoder',
                                                  TargetEncoder(),
                                                  [1, 10, 11])])),
                ('decision_tree',
                 <catboost.core.CatBoostClassifier object at 0x00000260342B0F70>)])

Сохраняем обученую модель.

In [26]:
filename = 'model_test.pkl'
pickle.dump(pipe, open(filename, 'wb'))

Рассчитаем ROC-AUC для обученной модели на тестовых данных.

In [27]:
print(f'ROC-AUC {roc_auc_score(y_test, pipe.predict_proba(X_test)[:, 1])}')

ROC-AUC 0.7834062362005785


# Строим таблицы для рекомендации постов

In [28]:
note = post_text.copy()

vectorizer = TfidfVectorizer()
F = vectorizer.fit_transform(note['text'])
note['text'] = F.toarray().max(axis=1)

In [29]:
columns = [f"text_{i}th_cluster" for i in range(1, n+1)]

pca = PCA(n_components=n, random_state=1)
pca_decomp = pca.fit_transform(F.toarray())

pca_decomp = pd.DataFrame(pca_decomp, columns=columns)

note = pd.concat((note, pca_decomp), axis=1)

In [30]:
note = pd.merge(note,
                post_stat,
                on='post_id',
                how='left')

Сохраняем таблицу note на сервер и сразу скачиваем её для проверки, что всё сохранилось как и задумывалось.

In [31]:
note.to_sql(
    "note_test_shaverdin",
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml",
    schema="public",
    if_exists='replace',
    index=False
)

note_shaverdin = pd.read_sql(
    """SELECT * FROM note_test_shaverdin""",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
note_shaverdin

,post_id,text,topic,text_1th_cluster,text_2th_cluster,text_3th_cluster,text_4th_cluster,text_5th_cluster,post_stat
0,1,0.439495,business,-0.118109,-0.202089,-0.092041,-0.111281,-0.090189,0.111436
1,2,0.290946,business,-0.142662,-0.224652,-0.000770,-0.071896,0.021130,0.078333
2,3,0.279045,business,-0.104280,-0.152657,-0.088950,-0.101965,-0.065593,0.117684
3,4,0.525321,business,-0.001531,-0.127432,-0.054627,-0.081059,-0.059621,0.125523
4,5,0.409826,business,0.055888,-0.104884,-0.004099,-0.015338,-0.053189,0.118426
...,...,...,...,...,...,...,...,...,...
7018,7315,0.226524,movie,-0.076983,0.210305,0.128307,0.041794,-0.085153,0.133903
7019,7316,0.333130,movie,-0.086584,0.131223,0.072691,-0.073864,-0.055744,0.093392
7020,7317,0.507582,movie,0.017890,0.105220,-0.016831,0.004615,0.001506,0.097027
7021,7318,0.263741,movie,-0.135336,0.119797,0.012075,0.015193,0.006627,0.091092


In [32]:
users = pd.merge(user_data,
                 user_stat,
                 on='user_id',
                 how='left')

Сохраняем таблицу users на сервер и сразу скачиваем её для проверки, что всё сохранилось как и задумывалось.

In [33]:
users.to_sql(
    "users_shaverdin",
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml",
    schema="public",
    if_exists='replace',
    index=False
)

users_shaverdin = pd.read_sql(
    """SELECT * FROM users_shaverdin""",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
users_shaverdin

,user_id,gender,age,country,city,exp_group,os,source,user_stat
0,200,1,34,Russia,Degtyarsk,3,Android,ads,0.107232
1,201,0,37,Russia,Abakan,0,Android,ads,0.077540
2,202,1,17,Russia,Smolensk,4,Android,ads,0.120166
3,203,0,18,Russia,Moscow,1,iOS,ads,0.159686
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads,0.142857
...,...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic,0.052356
163201,168549,0,18,Russia,Tula,2,Android,organic,0.080292
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic,0.093366
163203,168551,0,38,Russia,Moscow,3,iOS,organic,0.091429
